## Catastrophic cancellation

AMath 584, Autumn 2016

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


We start with a very simple computation: Adding 1 to a number and then subtracting 1.  Mathematically this should give us back what we started with, but maybe not in computer arithmetic.

Define $f(x) = x + 1 - 1$.  Mathematically this reduces to $f(x) = x$ so the condition number is 1.  Computationally if we implement this by first adding $x+1$ and storing it as a new variable and then subtracting 1 to get the result, we may lose many digits of accuracy if $x$ is very small. 

Try this with a small value:

In [2]:
x = pi * 1e-12
print '%16.15e' % x

3.141592653589793e-12


This has about 16 correct digits.

Now print out `x` as a fixed point number and compare to `y = x + 1.`:

In [3]:
print '%30.28f' % x
y = x + 1.
print '%30.28f' % y

0.0000000000031415926535897931
1.0000000000031414870704793429


Note that only the first 16 digits of `y` are correct (including the leading 1!)

So now if we subtract 1, we are left with garbage after the first 4 digits:

In [4]:
f = y - 1.
print 'Computed f = %16.15e' % f
print 'Original x = %16.15e' % x

Computed f = 3.141487070479343e-12
Original x = 3.141592653589793e-12


In [5]:
print 'The relative error is %6.3e' % (abs(f-x)/abs(x))

The relative error is 3.361e-05


## Error in evaluating the quadratic formula

The example above may seem silly, but exactly this sort of computation can easily arise even in simple practical examples.  As an example of catastrophic cancellation, consider solving for the two roots of a quadratic equation $ax^2 + bx + c = 0$ using the quadratic formula, which gives
$$
r_1 = \frac{-b - \sqrt{b^2 - 4ac}}{2a}, \qquad
r_2 = \frac{-b + \sqrt{b^2 - 4ac}}{2a}.
$$
Often this works fine, but consider the polynomial $x^2 + 2x + \epsilon$.  The root $r_2$ is very close to zero but the above formula computes it as 
$$
r_2 = -1 + \sqrt{1 - \epsilon}.
$$
We expect problems since Taylor series expansion of the square root shows that for small $\epsilon$ this is approximately
$$
r_2 = -1 + \left(1 - \frac 1 2 \epsilon + \frac 1 4 \epsilon^2 - \frac 3 8 \epsilon^3 + \cdots\right)
$$
The term in parentheses is very close to 1, and subtracting 1 from this gives $r_2$ very close to zero but we expect to lose many digits of accuracy in the process.

Note that we can get a very good approximation by instead cancelling the 1's before doing any computing and then truncating the series after a few terms, 
$$
r_2 \approx -\frac 1 2 \epsilon + \frac 1 4 \epsilon^2 - \frac 3 8 \epsilon^3
$$
We'll use this as the "exact solution" below to see how badly the evaluating the quadratic formula directly might do...

In [6]:
def quadratic_roots(a,b,c):
    """ 
    Find roots of ax^2 + bx + c by 
    standard quadratic formula.
    """
    d = sqrt(b**2 - 4.*a*c)
    r1 = (-b - d)/(2*a)
    r2 = (-b + d)/(2*a)
    return r1, r2

First find "exact solution" for a small value of $\epsilon$:

In [7]:
epsilon = 1e-12
r1_expansion = -2. + 0.5*epsilon - 0.25*epsilon**2 + 3./8. * epsilon**3
r2_expansion = -0.5*epsilon + 0.25*epsilon**2 - 3./8. * epsilon**3
print 'r1 obtained by expansion = %20.16e' % r1_expansion
print 'r2 obtained by expansion = %20.16e' % r2_expansion

r1 obtained by expansion = -1.9999999999995000e+00
r2 obtained by expansion = -4.9999999999974998e-13


Now test the quadratic formula:

In [8]:
r1,r2 = quadratic_roots(1., 2., epsilon)
r1_err = abs((r1-r1_expansion)/r1_expansion)
r2_err = abs((r2-r2_expansion)/r2_expansion)
print 'r1 obtained by quadratic formula = %20.16e,  relative error = %g' % (r1, r1_err)
print 'r2 obtained by quadratic formula = %20.16e,  relative error = %g' % (r2, r2_err)

r1 obtained by quadratic formula = -1.9999999999995000e+00,  relative error = 0
r2 obtained by quadratic formula = -5.0004445029117051e-13,  relative error = 8.89006e-05


The first root $r_1 \approx -2$ is calculated very accurately, but the small root has a large relative error due to cancellation.

### A more stable algorithm:

We can factor $ax^2 + bx + c = a(x-r_1)(x-r_2) = a(x^2 - (r_1+r_2)x + r_1r_2)$ which shows that $c = ar_1r_2$. So we can calculate $r_1$ accurately by the quadratic formula and then compute $r_2$ accurately by $r_2 = c / (ar_1) = \epsilon/r_1$.   In our case this yields:

In [9]:
r2_from_r1 = epsilon / r1
r2_err = abs((r2_from_r1-r2_expansion)/r2_expansion)
print 'r2 obtained from epsilon/r1 = %20.16e,  relative error = %g' % (r2_from_r1, r2_err)

r2 obtained from epsilon/r1 = -5.0000000000012500e-13,  relative error = 7.50036e-13


## Taylor series approximation of the exponential function

The Taylor series expansion of the exponential function $\exp(x) = e^x$ has partial sums
$$
S_n = \sum_{j=0}^n ~ \frac{x^j}{j!}
$$
Mathematically, this series converges for any complex number $x$, i.e. $S_n \rightarrow \exp(x)$ as $n \rightarrow \infty$.  (The number of terms needed to get a particular accuracy will of course grow for $|x|$ large.)

#### Code to compute this partial sum $S_n$:

Note that if $T_{j-1} = \frac{x^{j-1}}{(j-1)!}$ is term $j-1$ then the next term can be written as $T_j = \frac{x}{j} T_{j-1}$.  This is used in the code.

In [10]:
def taylor_exp(x, nterms, verbose=False):
    term = 1.
    partial_sum = term
    for j in range(1,nterms):
        term = term * x / j
        partial_sum = partial_sum + term
        if verbose:
            print 'x^%2i/%2i! = %25.16f, partial sum = %25.16f' \
                % (j,j,term,partial_sum)
    expx = exp(x)
    if verbose:
        print '                                Compare to exp(x) = %25.16f' % expx 
    return partial_sum

Experimenting with $x=15$ shows that $n=80$ is sufficient to get 16 digits of accuracy:

In [11]:
x = 15
f = taylor_exp(x,80)
expx = exp(x)
print "Computed value: %25.16f" % f
print "Correct value:  %25.16f" % expx

relative_error = abs(f-expx)/expx
print "Relative error = %g" % relative_error

Computed value:  3269017.3724721116013825
Correct value:   3269017.3724721106700599
Relative error = 2.84894e-16


But experimenting with $x=-15$ shows that no matter how large you take $n$ you can't get more than about 5 digits correct:

In [12]:
x = -15
f = taylor_exp(x,80)
expx = exp(x)
print "Computed value: %25.16f" % f
print "Correct value:  %25.16f" % expx

relative_error = abs(f-expx)/expx
print "Relative error = %g" % relative_error

Computed value:        0.0000003059094197
Correct value:         0.0000003059023205
Relative error = 2.32075e-05


To understand why, print out the terms and partial sum at each interation.  Note that the terms grow to be quite large.  Since each term in the series is the previous term multiplied by $x/j$, they continue to grow in magnitude up to $j = 15$. 

Since $x<0$, the terms alternate in sign.  The sum of all terms should be $\exp(-15) \approx 3\times 10^{-7}$, so these large terms must all nearly cancel out in the end.

But the large partial sums only have 16 digits of precision, most of which should cancel out.  There is not enough precision to capture what should be left over accurately.

This is a practical example of **catastrophic cancellation.**

In [13]:
f = taylor_exp(-15,80,verbose=True)

x^ 1/ 1! =      -15.0000000000000000, partial sum =      -14.0000000000000000
x^ 2/ 2! =      112.5000000000000000, partial sum =       98.5000000000000000
x^ 3/ 3! =     -562.5000000000000000, partial sum =     -464.0000000000000000
x^ 4/ 4! =     2109.3750000000000000, partial sum =     1645.3750000000000000
x^ 5/ 5! =    -6328.1250000000000000, partial sum =    -4682.7500000000000000
x^ 6/ 6! =    15820.3125000000000000, partial sum =    11137.5625000000000000
x^ 7/ 7! =   -33900.6696428571449360, partial sum =   -22763.1071428571449360
x^ 8/ 8! =    63563.7555803571449360, partial sum =    40800.6484375000000000
x^ 9/ 9! =  -105939.5926339285797440, partial sum =   -65138.9441964285797440
x^10/10! =   158909.3889508928696159, partial sum =    93770.4447544642898720
x^11/11! =  -216694.6212966721213888, partial sum =  -122924.1765422078315169
x^12/12! =   270868.2766208401299082, partial sum =   147944.1000786322983913
x^13/13! =  -312540.3191778924665414, partial sum =  -164596.219

### A more stable algorithm to compute $e^x$ for negative $x$

We can use the Taylor series to compute an accurate value if we use the fact that we can approximate $\exp(15)$ accurately from the Taylor series.   Then the value we want can be computed using $\exp(-15) = 1 / \exp(15)$.

Note that when $x=15$ the terms are the same magnitude as before but they are all positive, so there is no cancellation when they are added and the result is a large number approximating $\exp(15) \approx 3.27\times 10^6$:

In [14]:
x = -15
f_inverse = taylor_exp(-x,80,verbose=True)

x^ 1/ 1! =       15.0000000000000000, partial sum =       16.0000000000000000
x^ 2/ 2! =      112.5000000000000000, partial sum =      128.5000000000000000
x^ 3/ 3! =      562.5000000000000000, partial sum =      691.0000000000000000
x^ 4/ 4! =     2109.3750000000000000, partial sum =     2800.3750000000000000
x^ 5/ 5! =     6328.1250000000000000, partial sum =     9128.5000000000000000
x^ 6/ 6! =    15820.3125000000000000, partial sum =    24948.8125000000000000
x^ 7/ 7! =    33900.6696428571449360, partial sum =    58849.4821428571449360
x^ 8/ 8! =    63563.7555803571449360, partial sum =   122413.2377232142898720
x^ 9/ 9! =   105939.5926339285797440, partial sum =   228352.8303571428696159
x^10/10! =   158909.3889508928696159, partial sum =   387262.2193080357392319
x^11/11! =   216694.6212966721213888, partial sum =   603956.8406047078315169
x^12/12! =   270868.2766208401299082, partial sum =   874825.1172255480196327
x^13/13! =   312540.3191778924665414, partial sum =  1187365.436

Now when we invert the value computed above, we get a good approximation to $\exp(-15)$:

In [15]:
f = 1./f_inverse
expx = exp(x)
print "Computed value: %27.25f" % f
print "Correct value:  %27.25f" % expx

relative_error = abs(f-exp(x))/exp(x)
print "Relative error = %g" % relative_error

Computed value: 0.0000003059023205018256800
Correct value:  0.0000003059023205018257859
Relative error = 3.46121e-16
